# ●データ拡張
学習データが少ないと過学習が発生し、汎化性能が低下してしまいます。  
しかしながら、多くの画像などの学習データを集めるのには大きな手間がかかります。  
この問題への対策の一つが**データ拡張**です。  
データ拡張は、画像に反転、拡大、縮小などの変換を加えることで、学習データの「水増し」を行います。  
これにより、学習データ不足の問題が解消されることがあります。

## ●データ拡張の実装
KerasのImageDataGeneratorを使ってデータ拡張を行います。  
今回は、cifar-10の画像に回転を加えたものをいくつか生成します。  
画像を表示するコードは、再利用のため関数にまとめておきます。  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator

(x_train, t_train), (x_test, t_test) = cifar10.load_data()

image = x_train[12]
plt.imshow(image)
plt.show()

def show_images(image, generator):
    channel, height, width = image.shape
    image = image.reshape(1, channel, height, width)  # バッチ対応
    gen = generator.flow(image, batch_size=1)  # 変換された画像の生成

    plt.figure(figsize=(9, 9))
    for i in range(9):
        gen_img = gen.next()[0].astype(np.uint8)  # 画像の取得
        plt.subplot(3, 3, i + 1)
        plt.imshow(gen_img)
    plt.show()

# -20°から20°の範囲でランダムに回転を行う画像生成器
generator = ImageDataGenerator(rotation_range=20)
show_images(image, generator)

## ●様々なデータ拡張
width_shift_rangeを指定することにより、画像を水平方向にシフトすることができます。

In [ ]:
generator = ImageDataGenerator(width_shift_range=0.5)  # 画像サイズの半分の範囲でランダムにシフトする
show_images(image, generator)

height_shift_rangeを指定することにより、画像を垂直方向にシフトすることができます。

In [ ]:
generator = ImageDataGenerator(height_shift_range=0.5)  # 画像サイズの半分の範囲でランダムにシフトする
show_images(image, generator)

shear_rangeによりシアー強度の範囲を指定すると、引っ張るような変換を加えた画像を生成することができます。

In [ ]:
generator = ImageDataGenerator(shear_range=20)  # シアー強度の範囲を指定
show_images(image, generator)

zoom_rangeで拡大縮小する範囲を指定することができます。  
この範囲内で、水平方向、垂直方向それぞれの拡大率がランダムに指定されます。

In [ ]:
generator = ImageDataGenerator(zoom_range=0.4)  # 拡大縮小する範囲を指定
show_images(image, generator)

horizontal_flipとvertical_flipで、水平及び垂直方向に反転することができます。

In [ ]:
generator = ImageDataGenerator(horizontal_flip=True, vertical_flip=True)  # 水平、垂直方向にランダムに反転
show_images(image, generator)

## ●CNNへの適用
CNNの入力をデータ拡張します。  
使用するモデルは、以前に構築したものと同じです。

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam

batch_size = 32
epochs = 20
n_class = 10

t_train = keras.utils.to_categorical(t_train, n_class)  # one-hot表現に
t_test = keras.utils.to_categorical(t_test, n_class)

model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))  # ゼロパディング、バッチサイズ以外の画像の形状を指定
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # 一次元の配列に変換
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))  # ドロップアウト
model.add(Dense(n_class))
model.add(Activation('softmax'))

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

## ●学習
モデルを訓練します。  
ImageDataGeneratorを使用する際は、学習にfit_generatorメソッドを使います。  
学習には時間がかかりますので、編集→ノートブックの設定のハードウェアアクセラレーターでGPUを選択しましょう。

In [ ]:
x_train = x_train / 255  # 0から1の範囲に収める
x_test = x_test / 255

generator = ImageDataGenerator(
           rotation_range=0.2,
           horizontal_flip=True)
generator.fit(x_train)

history = model.fit_generator(generator.flow(x_train, t_train, batch_size=batch_size),
                              epochs=epochs,
                              validation_data=(x_test, t_test))

## ●学習の推移
学習の推移を表示します。  
データ拡張をしていない場合と比較して、汎化性能が向上していることを確認しましょう。

In [ ]:
import matplotlib.pyplot as plt

train_loss = history.history['loss']  # 訓練用データの誤差
train_acc = history.history['accuracy']  # 訓練用データの精度
val_loss = history.history['val_loss']  # 検証用データの誤差
val_acc = history.history['val_accuracy']  # 検証用データの精度

plt.plot(np.arange(len(train_loss)), train_loss, label='loss')
plt.plot(np.arange(len(val_loss)), val_loss, label='val_loss')
plt.legend()
plt.show()

plt.plot(np.arange(len(train_acc)), train_acc, label='acc')
plt.plot(np.arange(len(val_acc)), val_acc, label='val_acc')
plt.legend()
plt.show()

## ●評価
モデルの評価を行います。  

In [ ]:
loss, accuracy = model.evaluate(x_test, t_test)
print(loss, accuracy)